This notebook contain methods to dynamically take a JSON Object and normalise it. 
This includes 
* Flattening nested objects
* Extracting arrays into seperate objects

# Example Code From Internet - Loop through JSON elements, flatten all into one flat structure. 
Link: https://gist.github.com/shreyasms17/96f74e45d862f8f1dce0532442cc95b2
Benifits: 
1. Uses standard code
2. Will flatten arrays
3. Will flatten nested attributes

Drawbacks
1. Arrays cause parent attributes to duplicate

In [0]:

from pyspark.sql.functions import col, explode_outer, from_json, lit, concat
from pyspark.sql.types import StructType, ArrayType


def get_json_df(input_df, primary_partition_column, json_column_name, spark_session):
    '''
    Description:
    This function provides the schema of json records and the dataframe to be used for flattening
        :param input_df: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :primary_partition_column: [type: string] name of primary partition column
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return df: dataframe to be used for flattening
    '''
    input_df = input_df if primary_partition_column is None else input_df.drop(primary_partition_column)
    # creating a column transformedJSON to create an outer struct
    df1 = input_df.withColumn('transformed_json', concat(lit("""{"transformedJSON" :"""), input_df[json_column_name], lit("""}""")))
    json_df = spark_session.read.json(df1.rdd.map(lambda row: row.transformed_json))
    # get schema
    json_schema = json_df.schema
    df = df1.drop(json_column_name).withColumn(json_column_name, from_json(col('transformed_json'), json_schema)).drop('transformed_json').select(f'{json_column_name}.*', '*').drop(json_column_name)
    return df


def execute_autoflatten(df, json_column_name):
    '''
    Description:
    This function executes the core autoflattening operation
        :param df: [type: pyspark.sql.dataframe.DataFrame] dataframe to be used for flattening
        :param json_column_name: [type: string] name of the column with json string
        :return df: DataFrame containing flattened records
    '''
    # gets all fields of StructType or ArrayType in the nested_fields dictionary
    nested_fields = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)
    ])

    # repeat until all nested_fields i.e. belonging to StructType or ArrayType are covered
    while nested_fields:
        # if there are any elements in the nested_fields dictionary
        if nested_fields:
            # get a column
            column_name = list(nested_fields.keys())[0]
            # if field belongs to a StructType, all child fields inside it are accessed
            # and are aliased with complete path to every child field
            if isinstance(nested_fields[column_name], StructType):
                unnested = [col(column_name + '.' + child).alias(column_name + '>' + child) for child in [ n.name for n in  nested_fields[column_name]]]
                df = df.select("*", *unnested).drop(column_name)
            # else, if the field belongs to an ArrayType, an explode_outer is done
            elif isinstance(nested_fields[column_name], ArrayType):
                df = df.withColumn(column_name, explode_outer(column_name))

        # Now that df is updated, gets all fields of StructType and ArrayType in a fresh nested_fields dictionary
        nested_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)
        ])

    # renaming all fields extracted with json> to retain complete path to the field
    for df_col_name in df.columns:
        df = df.withColumnRenamed(df_col_name, df_col_name.replace("transformedJSON", json_column_name))
    return df



def execute(input_df, primary_partition_column, json_column_name, spark_session):
    '''
    Description:
    This function executes the flattening of json records in the dataframe
        :param input_df: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :primary_partition_column: [type: string] name of primary partition column
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return unstd_df: contains flattened dataframe with unstandardized column name format
    '''
    json_df = get_json_df(input_df, primary_partition_column, json_column_name, spark_session)
    unstd_df = execute_autoflatten(json_df, json_column_name)
    return unstd_df

In [0]:
#Get raw data into a dataframe
#Note that by default, JSON files with values scattered over multiple lines support is set to false -- so use the .option("multiline","true") to enable that
rawJsonDF_JsonFile = spark.read\
.format("json")\
.option("multiline","true")\
.load("/mnt/datalake_rawdata/json-generator.com/generated.json")

#rawJsonDF_JsonFile.printSchema()

display(rawJsonDF_JsonFile)

NestedAttributesTest,_id,about,address,age,balance,company,email,eyeColor,favoriteFruit,friends,gender,greeting,guid,index,isActive,latitude,longitude,name,phone,picture,registered,tags
"List(5, 4, 1, 6, 3, 2)",622b43c868725fca32d05127,Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.,"116 Stewart Street, Longoria, Alaska, 3997",20,"$2,387.61",GENMY,hughesslater@genmy.com,green,banana,"List(List(0, Katie Carey), List(1, Roth Gould), List(2, Trisha Lindsay))",male,"Hello, Hughes Slater! You have 1 unread messages.",8a81ca3a-0813-4902-820d-d58e9703ec04,0,true,-47.586921,22.966975,Hughes Slater,+1 (883) 494-2174,http://placehold.it/32x32,2022-02-03T10:33:31 -02:00,"List(non, ex, sint, culpa, aliqua, esse, pariatur)"
"List(5, 4, 1, 6, 3, 2)",622b43c8d2984c9956b839d9,Sunt culpa commodo ullamco aliqua fugiat sit fugiat eu fugiat eiusmod amet esse. Consequat nostrud exercitation deserunt magna labore elit ipsum Lorem laboris cupidatat reprehenderit mollit magna velit. Duis esse cillum proident ad id excepteur. Dolore excepteur cillum duis commodo. Sint dolore voluptate in amet nostrud culpa qui consequat tempor incididunt eiusmod irure sit officia. Consequat cillum occaecat Lorem nulla nisi qui excepteur.,"894 Cambridge Place, Dexter, Utah, 8622",25,"$1,910.43",GEEKNET,burchconrad@geeknet.com,green,apple,"List(List(0, Charles Newman), List(1, Maude Robinson), List(2, Taylor Goodwin))",male,"Hello, Burch Conrad! You have 7 unread messages.",f4b512ca-e362-48a5-a5a5-1b21013bbd4c,1,false,33.526889,-69.444991,Burch Conrad,+1 (863) 523-2711,http://placehold.it/32x32,2018-10-11T07:17:58 -02:00,"List(in, non, adipisicing, veniam, irure, excepteur, laboris)"
"List(5, 4, 1, 6, 3, 2)",622b43c806bb45d49234240b,Lorem aute eiusmod et Lorem quis ea. Duis velit anim minim aute magna eiusmod excepteur nulla ad velit nostrud. Amet laboris voluptate nostrud nulla do minim non mollit duis sunt officia sunt nisi reprehenderit.,"111 Beverly Road, Canby, Oklahoma, 4837",24,"$1,711.22",SKINSERVE,ellapacheco@skinserve.com,brown,apple,"List(List(0, Montoya Becker), List(1, Reilly Phelps), List(2, Reeves Tran))",female,"Hello, Ella Pacheco! You have 6 unread messages.",6f752038-214b-4cd7-b6d0-14250292a1e3,2,false,-18.438374,-137.932995,Ella Pacheco,+1 (915) 430-3434,http://placehold.it/32x32,2015-08-13T05:28:29 -02:00,"List(duis, occaecat, Lorem, aute, ut, nisi, enim)"
"List(5, 4, 1, 6, 3, 2)",622b43c8858495221ea6ecda,Sit non veniam amet minim. Sint cillum excepteur et incididunt. Do reprehenderit ex elit ut excepteur nostrud. Commodo elit minim velit sint esse duis eu sint minim laborum mollit. Fugiat nulla consequat veniam nisi culpa sit do veniam. Sunt anim est culpa nisi anim mollit consequat. Qui dolor sint id minim irure cillum.,"372 Ainslie Street, Cornfields, Alabama, 6363",23,"$1,889.73",EBIDCO,estellarich@ebidco.com,brown,strawberry,"List(List(0, Walker William), List(1, Kathrine Alexander), List(2, Faye Hoffman))",female,"Hello, Estella Rich! You have 7 unread messages.",121b43bf-abc7-46bf-8ac3-5355fe1d8290,3,true,38.887249,-41.450973,Estella Rich,+1 (957) 478-3558,http://placehold.it/32x32,2018-03-04T12:10:29 -02:00,"List(cillum, occaecat, et, pariatur, duis, adipisicing, nostrud)"
"List(5, 4, 1, 6, 3, 2)",622b43c876311a36c1c9da33,Laborum eu deserunt commodo deserunt pariatur sit quis fugiat consequat cillum ex mollit dolore consequat. Officia irure cillum et do incididunt eiusmod laborum consequat laboris ex fugiat ullamco. Anim nisi velit adipisicing consectetur Lorem ut cupidatat.,"521 Stratfo

In [0]:
#Get raw data into a dataframe
#Ensure you specify the quote and esacpe characters. 
#Multiple is false by default. 
rawJsonDF = spark.read\
.format("csv")\
.options(header="true", escape='"', quote='"', multiline=False, inferSchema=True)\
.load("/mnt/datalake_rawdata/json-generator.com/MockJsonInCSV.csv")

display(rawJsonDF)



RowNumber JSONColumn 1 [ { "_id": "622b43c868725fca32d05127", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 0, "guid": "8a81ca3a-0813-4902-820d-d58e9703ec04", "isActive": true, "balance": "$2,387.61", "picture": "http://placehold.it/32x32", "age": 20, "eyeColor": "green", "name": "Hughes Slater", "gender": "male", "company": "GENMY", "email": "hughesslater@genmy.com", "phone": "+1 (883) 494-2174", "address": "116 Stewart Street, Longoria, Alaska, 3997", "about": "Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.\r\n", "registered": "2022-02-03T10:33:31 -02:00", "latitude": -47.586921, "longitude": 22.966975, "tags": [ "non", "ex", "sint", "culpa", "aliqua", "esse", "pariatur" ], "friends": [ { "id": 0, "name": "Katie Carey" }, { "id": 1, "name": "Roth Gould" }, { "id": 2, "name": "Trisha Lindsay" } ], "greeting": "Hello, Hughes Slater! You have 1 unread messages.", "favoriteFruit": "banana" }, { "_id": "622b43c8d2984c9956b839d9", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 1, "guid": "f4b512ca-e362-48a5-a5a5-1b21013bbd4c", "isActive": false, "balance": "$1,910.43", "picture": "http://placehold.it/32x32", "age": 25, "eyeColor": "green", "name": "Burch Conrad", "gender": "male", "company": "GEEKNET", "email": "burchconrad@geeknet.com", "phone": "+1 (863) 523-2711", "address": "894 Cambridge Place, Dexter, Utah, 8622", "about": "Sunt culpa commodo ullamco aliqua fugiat sit fugiat eu fugiat eiusmod amet esse. Consequat nostrud exercitation deserunt magna labore elit ipsum Lorem laboris cupidatat reprehenderit mollit magna velit. Duis esse cillum proident ad id excepteur. Dolore excepteur cillum duis commodo. Sint dolore voluptate in amet nostrud culpa qui consequat tempor incididunt eiusmod irure sit officia. Consequat cillum occaecat Lorem nulla nisi qui excepteur.\r\n", "registered": "2018-10-11T07:17:58 -02:00", "latitude": 33.526889, "longitude": -69.444991, "tags": [ "in", "non", "adipisicing", "veniam", "irure", "excepteur", "laboris" ], "friends": [ { "id": 0, "name": "Charles Newman" }, { "id": 1, "name": "Maude Robinson" }, { "id": 2, "name": "Taylor Goodwin" } ], "greeting": "Hello, Burch Conrad! You have 7 unread messages.", "favoriteFruit": "apple" }, { "_id": "622b43c806bb45d49234240b", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 2, "guid": "6f752038-214b-4cd7-b6d0-14250292a1e3", "isActive": false, "balance": "$1,711.22", "picture": "http://placehold.it/32x32", "age": 24, "eyeColor": "brown", "name": "Ella Pacheco", "gender": "female", "company": "SKINSERVE", "email": "ellapacheco@skinserve.com", "phone": "+1 (915) 430-3434", "address": "111 Beverly Road, Canby, Oklahoma, 4837", "about": "Lorem aute eiusmod et Lorem quis ea. Duis velit anim minim aute magna eiusmod excepteur nulla ad velit nostrud. Amet laboris voluptate nostrud nulla do minim non mollit duis sunt officia sunt nisi reprehenderit.\r\n", "registered": "2015-08-13T05:28:29 -02:00", "latitude": -18.438374, "longitude": -137.932995, "tags": [ "duis", "occaecat", "Lorem", "aute", "ut", "nisi", "enim" ], "friends": [ { "id": 0, "name": "Montoy

In [0]:
#use the first function in the sequence
get_json_df_result = get_json_df(rawJsonDF, None, "JSONColumn", spark)

display(get_json_df_result)

get_json_df_result.printSchema()

transformedJSON RowNumber List(List(List(5, 4, 1, 6, 3, 2), 622b43c868725fca32d05127, Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.
, 116 Stewart Street, Longoria, Alaska, 3997, 20, $2,387.61, GENMY, hughesslater@genmy.com, green, banana, List(List(0, Katie Carey), List(1, Roth Gould), List(2, Trisha Lindsay)), male, Hello, Hughes Slater! You have 1 unread messages., 8a81ca3a-0813-4902-820d-d58e9703ec04, 0, true, -47.586921, 22.966975, Hughes Slater, +1 (883) 494-2174, http://placehold.it/32x32, 2022-02-03T10:33:31 -02:00, List(non, ex, sint, culpa, aliqua, esse, pariatur)), List(List(5, 4, 1, 6, 3, 2), 622b43c8d2984c9956b839d9, Sunt culpa commodo ullamco aliqua fugiat sit fugiat eu fugiat eiusmod amet esse. Consequat nostrud exercitation deserunt magna labore elit ipsum Lorem laboris cupidatat reprehenderit mollit magna velit. Duis esse cillum proident ad id excepteur. Dolore excepteur cillum duis commodo. Sint dolore voluptate in amet nostrud culpa qui consequat tempor incididunt eiusmod irure sit officia. Consequat cillum occaecat Lorem nulla nisi qui excepteur.
, 894 Cambridge Place, Dexter, Utah, 8622, 25, $1,910.43, GEEKNET, burchconrad@geeknet.com, green, apple, List(List(0, Charles Newman), List(1, Maude Robinson), List(2, Taylor Goodwin)), male, Hello, Burch Conrad! You have 7 unread messages., f4b512ca-e362-48a5-a5a5-1b21013bbd4c, 1, false, 33.526889, -69.444991, Burch Conrad, +1 (863) 523-2711, http://placehold.it/32x32, 2018-10-11T07:17:58 -02:00, List(in, non, adipisicing, veniam, irure, excepteur, laboris)), List(List(5, 4, 1, 6, 3, 2), 622b43c806bb45d49234240b, Lorem aute eiusmod et Lorem quis ea. Duis velit anim minim aute magna eiusmod excepteur nulla ad velit nostrud. Amet laboris voluptate nostrud nulla do minim non mollit duis sunt officia sunt nisi reprehenderit.
, 111 Beverly Road, Canby, Oklahoma, 4837, 24, $1,711.22, SKINSERVE, ellapacheco@skinserve.com, brown, apple, List(List(0, Montoya Becker), List(1, Reilly Phelps), List(2, Reeves Tran)), female, Hello, Ella Pacheco! You have 6 unread messages., 6f752038-214b-4cd7-b6d0-14250292a1e3, 2, false, -18.438374, -137.932995, Ella Pacheco, +1 (915) 430-3434, http://placehold.it/32x32, 2015-08-13T05:28:29 -02:00, List(duis, occaecat, Lorem, aute, ut, nisi, enim)), List(List(5, 4, 1, 6, 3, 2), 622b43c8858495221ea6ecda, Sit non veniam amet minim. Sint cillum excepteur et incididunt. Do reprehenderit ex elit ut excepteur nostrud. Commodo elit minim velit sint esse duis eu sint minim laborum mollit. Fugiat nulla consequat veniam nisi culpa sit do veniam. Sunt anim est culpa nisi anim mollit consequat. Qui dolor sint id minim irure cillum.
, 372 Ainslie Street, Cornfields, Alabama, 6363, 23, $1,889.73, EBIDCO, estellarich@ebidco.com, brown, strawberry, List(List(0, Walker William), List(1, Kathrine Alexander), List(2, Faye Hoffman)), female, Hello, Estella Rich! You have 7 unread messages., 121b43bf-abc7-46bf-8ac3-5355fe1d8290, 3, true, 38.887249, -41.450973, Estella Rich, +1 (957) 478-3558, http://placehold.it/32x32, 2018-03-04T12:10:29 -02:00, List(cillum, occaecat, et, pariatur, duis, adipisicing, nostrud)), List(List(5, 4, 1, 6, 3, 2), 622b43c876311a36c1c9da33, Laborum eu deserunt commodo deserunt pariatur sit quis fugiat consequat cillum ex mollit dolore consequat. Officia irure cillum et do incididunt eiusmod laborum consequat laboris ex fugiat ullamco. Anim nisi velit adipisicing consectetur Lorem ut cupidatat.
, 521 Stratford Road, Stewartville, Vermont, 6312, 33, $1,498.87, LOTRON, carpentersu

root
-- transformedJSON: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- NestedAttributesTest: struct (nullable = true)
 | | |-- NestedAttributeFive: string (nullable = true)
 | | |-- NestedAttributeFour: string (nullable = true)
 | | |-- NestedAttributeOne: string (nullable = true)
 | | |-- NestedAttributeSix: string (nullable = true)
 | | |-- NestedAttributeThree: string (nullable = true)
 | | |-- NestedAttributeTwo: string (nullable = true)
 | |-- _id: string (nullable = true)
 | |-- about: string (nullable = true)
 | |-- address: string (nullable = true)
 | |-- age: long (nullable = true)
 | |-- balance: string (nullable = true)
 | |-- company: string (nullable = true)
 | |-- email: string (nullable = true)
 | |-- eyeColor: string (nullable = true)
 | |-- favoriteFruit: string (nullable = true)
 | |-- friends: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- id: long (nullable = true)
 | | | |-- name: string (nullable = true)
 | |-- gender: string (nullable = true)
 | |-- greeting: string (nullable = true)
 | |-- guid: string (nullable = true)
 | |-- index: long (nullable = true)
 | |-- isActive: boolean (nullable = true)
 | |-- latitude: double (nullable = true)
 | |-- longitude: double (nullable = true)
 | |-- name: string (nullable = true)
 | |-- phone: string (nullable = true)
 | |-- picture: string (nullable = true)
 | |-- registered: string (nullable = true)
 | |-- tags: array (nullable = true)
 | | |-- element: string (containsNull = true)
-- RowNumber: integer (nullable = true)

In [0]:
autoFlattenReturnDF = execute_autoflatten(get_json_df_result, "transformedJSON")

display(autoFlattenReturnDF)

RowNumber,transformedJSON>_id,transformedJSON>about,transformedJSON>address,transformedJSON>age,transformedJSON>balance,transformedJSON>company,transformedJSON>email,transformedJSON>eyeColor,transformedJSON>favoriteFruit,transformedJSON>gender,transformedJSON>greeting,transformedJSON>guid,transformedJSON>index,transformedJSON>isActive,transformedJSON>latitude,transformedJSON>longitude,transformedJSON>name,transformedJSON>phone,transformedJSON>picture,transformedJSON>registered,transformedJSON>tags,transformedJSON>NestedAttributesTest>NestedAttributeFive,transformedJSON>NestedAttributesTest>NestedAttributeFour,transformedJSON>NestedAttributesTest>NestedAttributeOne,transformedJSON>NestedAttributesTest>NestedAttributeSix,transformedJSON>NestedAttributesTest>NestedAttributeThree,transformedJSON>NestedAttributesTest>NestedAttributeTwo,transformedJSON>friends>id,transformedJSON>friends>name
1,622b43c868725fca32d05127,Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.,"116 Stewart Street, Longoria, Alaska, 3997",20,"$2,387.61",GENMY,hughesslater@genmy.com,green,banana,male,"Hello, Hughes Slater! You have 1 unread messages.",8a81ca3a-0813-4902-820d-d58e9703ec04,0,true,-47.586921,22.966975,Hughes Slater,+1 (883) 494-2174,http://placehold.it/32x32,2022-02-03T10:33:31 -02:00,non,5,4,1,6,3,2,0,Katie Carey
1,622b43c868725fca32d05127,Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.,"116 Stewart Street, Longoria, Alaska, 3997",20,"$2,387.61",GENMY,hughesslater@genmy.com,green,banana,male,"Hello, Hughes Slater! You have 1 unread messages.",8a81ca3a-0813-4902-820d-d58e9703ec04,0,true,-47.586921,22.966975,Hughes Slater,+1 (883) 494-2174,http://placehold.it/32x32,2022-02-03T10:33:31 -02:00,ex,5,4,1,6,3,2,0,Katie Carey
1,622b43c868725fca32d05127,Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.,"116 Stewart Street, Longoria, Alaska, 3997",20,"$2,387.61",GENMY,hughesslater@genmy.com,green,banana,male,"Hello, Hughes Slater! You have 1 unread messages.",8a81ca3a-0813-4902-820d-d58e9703ec04,0,true,-47.586921,22.966975,Hughes Slater,+1 (883) 494-2174,http://placehold.it/32x32,2022-02-03T10:33:31 -02:00,sint,5,4,1,6,3,2,0,Katie Carey
1,622b43c868725fca32d05127,Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea

In [0]:
autoFlattenReturnDF = execute_autoflatten(rawJsonDF, "JSONColumn")

display(autoFlattenReturnDF)

RowNumber JSONColumn 1 [ { "_id": "622b43c868725fca32d05127", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 0, "guid": "8a81ca3a-0813-4902-820d-d58e9703ec04", "isActive": true, "balance": "$2,387.61", "picture": "http://placehold.it/32x32", "age": 20, "eyeColor": "green", "name": "Hughes Slater", "gender": "male", "company": "GENMY", "email": "hughesslater@genmy.com", "phone": "+1 (883) 494-2174", "address": "116 Stewart Street, Longoria, Alaska, 3997", "about": "Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.\r\n", "registered": "2022-02-03T10:33:31 -02:00", "latitude": -47.586921, "longitude": 22.966975, "tags": [ "non", "ex", "sint", "culpa", "aliqua", "esse", "pariatur" ], "friends": [ { "id": 0, "name": "Katie Carey" }, { "id": 1, "name": "Roth Gould" }, { "id": 2, "name": "Trisha Lindsay" } ], "greeting": "Hello, Hughes Slater! You have 1 unread messages.", "favoriteFruit": "banana" }, { "_id": "622b43c8d2984c9956b839d9", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 1, "guid": "f4b512ca-e362-48a5-a5a5-1b21013bbd4c", "isActive": false, "balance": "$1,910.43", "picture": "http://placehold.it/32x32", "age": 25, "eyeColor": "green", "name": "Burch Conrad", "gender": "male", "company": "GEEKNET", "email": "burchconrad@geeknet.com", "phone": "+1 (863) 523-2711", "address": "894 Cambridge Place, Dexter, Utah, 8622", "about": "Sunt culpa commodo ullamco aliqua fugiat sit fugiat eu fugiat eiusmod amet esse. Consequat nostrud exercitation deserunt magna labore elit ipsum Lorem laboris cupidatat reprehenderit mollit magna velit. Duis esse cillum proident ad id excepteur. Dolore excepteur cillum duis commodo. Sint dolore voluptate in amet nostrud culpa qui consequat tempor incididunt eiusmod irure sit officia. Consequat cillum occaecat Lorem nulla nisi qui excepteur.\r\n", "registered": "2018-10-11T07:17:58 -02:00", "latitude": 33.526889, "longitude": -69.444991, "tags": [ "in", "non", "adipisicing", "veniam", "irure", "excepteur", "laboris" ], "friends": [ { "id": 0, "name": "Charles Newman" }, { "id": 1, "name": "Maude Robinson" }, { "id": 2, "name": "Taylor Goodwin" } ], "greeting": "Hello, Burch Conrad! You have 7 unread messages.", "favoriteFruit": "apple" }, { "_id": "622b43c806bb45d49234240b", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 2, "guid": "6f752038-214b-4cd7-b6d0-14250292a1e3", "isActive": false, "balance": "$1,711.22", "picture": "http://placehold.it/32x32", "age": 24, "eyeColor": "brown", "name": "Ella Pacheco", "gender": "female", "company": "SKINSERVE", "email": "ellapacheco@skinserve.com", "phone": "+1 (915) 430-3434", "address": "111 Beverly Road, Canby, Oklahoma, 4837", "about": "Lorem aute eiusmod et Lorem quis ea. Duis velit anim minim aute magna eiusmod excepteur nulla ad velit nostrud. Amet laboris voluptate nostrud nulla do minim non mollit duis sunt officia sunt nisi reprehenderit.\r\n", "registered": "2015-08-13T05:28:29 -02:00", "latitude": -18.438374, "longitude": -137.932995, "tags": [ "duis", "occaecat", "Lorem", "aute", "ut", "nisi", "enim" ], "friends": [ { "id": 0, "name": "Montoy

# Tangent Version - Doesn't write data to target
**NOTE - this version can only cater for one level of nested arrays. Not multiple levels of nested arrays. **

Considerations
1. The column that has the JSON data must contain an array in each record.

What these functions can cater for
* Receive a dataframe with a column with JSON data
* it expects the top level to be an array of JSON records. It will explode this once. 
  * So each array in each row of the dataframe will be exploded as each row contains an array. 
  * At this point, each array record is a new record in the dataframe, but structs and nested arrays remain unchanged. No data duplication will occur at this level. 
  * If it is not an array, dont explode and move on to the next step
* Then expand all the struct fields to flatten it - this wont cause any data duplication. Struct fields are nested scalar attributes.
* Write the parent level attributes and flattened struct attributes to a target. 
* For each array column remaining, loop through them again and recursively call this function to write that array to a target as well

TODO
* [TODO] - change the function execute_autoflatten_tangent to write to file path in the lake to check the final output
* [TODO] - need to figure out how the verion will look if we dont expect nested arrays. If it is just flat array of json records. DO we still use this, or a simpler method? 
* [TODO] - How do we implement this in the silver layer - IDEA - have the function write to parquet files instead of delta tables. Still in the silver layer. THEN have the function build up a list of the files it produces and return that list. Then the rest of the silver notebook can process each flat silver file produced in sequence - OR we can make a new silver notebook per file produced.

In [0]:
#Get raw data into a dataframe
#Ensure you specify the quote and esacpe characters. 
#Multiple is false by default. 

#Ensure only the column with the JSON data is selected to have the downstream code work
rawJsonDF = spark.read\
.format("csv")\
.options(header="true", escape='"', quote='"', multiline=False, inferSchema=True)\
.load("/mnt/datalake_rawdata/json-generator.com/MockJsonInCSV.csv")\
.select("JSONColumn")

display(rawJsonDF)


JSONColumn


In [0]:

from pyspark.sql.functions import col, explode_outer, from_json, lit, concat, explode, array
from pyspark.sql.types import StructType, ArrayType


def get_json_df_tangent(inputDF, primary_partition_column, json_column_name, spark_session):
    '''
    Description:
    This function provides the schema of json records and the dataframe to be used for flattening. If this doesnt happen, the source JSON String remains a string and cant be queries like JSON
        :param inputDF: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :primary_partition_column: [type: string] name of primary partition column
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return df: dataframe to be used for flattening
    '''
    inputDF = inputDF if primary_partition_column is None else inputDF.drop(primary_partition_column)
    # creating a column transformedJSON to create an outer struct
    df1 = inputDF.withColumn('transformed_json', concat(lit("""{"transformed_json" :"""), inputDF[json_column_name], lit("""}""")))
    json_df = spark_session.read.json(df1.rdd.map(lambda row: row.transformed_json))
    # get schema
    json_schema = json_df.schema
    
    #Return a dataframe with the orignal column name but with proper JSON typed data
    df = df1.drop(json_column_name)\
        .withColumn(json_column_name, from_json(col('transformed_json'), json_schema))\
        .drop('transformed_json')\
        .select(f'{json_column_name}.*', '*')\
        .drop(json_column_name)\
        .withColumnRenamed("transformed_json", json_column_name)
    return df

def execute_autoflatten_tangent(df, json_column_name, json_key_field_name):
    '''
    Description:
    This function executes the core autoflattening operation
        :param df: [type: pyspark.sql.dataframe.DataFrame] dataframe to be used for flattening - contains at least one column with JSON data
        :param json_column_name: [type: string] name of the column with json string
        :json_key_field_name: [type: string] Name of the JSON Field that is the key of the document. Is used for nested arrays that are extracted to relate back to the primary dataset. 
            Unique per row of the df dataframe
            Should be in the format of "json_column_name>json_key_field_name" e.g. "JSONCOlumn>id"
        :return df: 
    '''
    ######################################################################################
    # Check if the JSON data in the json_column_name column in each record of the df data frame is an array type. 
        # If so, explode the array once to get a flat structure with parent scalar attributes, and Structs (nested scalar attributes) and Arrays still intact. 
        # No data duplication will occur at this level. 
    ######################################################################################
    if isinstance(df.schema[json_column_name].dataType, ArrayType):
        #Keep this here permanently, for run time troubleshooting, even in prod
        print("json_column_name " + json_column_name + " is array type")
        
        #Since the entire json_column_name is an array, explode it once
        df = df.withColumn(json_column_name, explode_outer(json_column_name))
    else:
        print("json_column_name " + json_column_name + " is NOT array type: ", df.schema[json_column_name].dataType)
        
    ######################################################################################
    # Get all fields of StructType in a nested_fields dictionary. 
    # The next step will loop through these columns and expand them to get a flat structure. 
    # The parent attribute that contained the nested structure will be prefixed to the column name like Person>Name
    # This will not cause data duplication
    ######################################################################################    
    nested_fields = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if isinstance(field.dataType, StructType)
    ])
    
    ######################################################################################
    # For each struct type field in nested_fields, flatten them. 
    # Repeat until all nested_fields are covered
    ######################################################################################    
    while nested_fields:
        # check if there are any elements in the nested_fields dictionary
        if nested_fields:
            # get a column name to process
            column_name = list(nested_fields.keys())[0]
            # if field belongs to a StructType, all child fields inside it are accessed
            # and are aliased with complete path to every child field
            if isinstance(nested_fields[column_name], StructType):
                unnested = [col(column_name + '.' + child).alias(column_name + '>' + child) for child in [ n.name for n in  nested_fields[column_name]]]
                df = df.select("*", *unnested).drop(column_name)
            
        # Now that df is updated with the above column being flattened, get a new list of the fields of StructType in a fresh nested_fields dictionary
        # so that the next while loop can check again. 
        nested_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, StructType) # or isinstance(field.dataType, ArrayType)
        ])
    
    ######################################################################################
    # At this point, the top level structs have been flattened, but there are still nested arrays. 
    # Need to take non-array fields list, write that data to a target and only keep the remaining array fields with the key field used for related the data back to the primary data
    # For each remaining array field, do the above exactly the same again
    ######################################################################################
    # Gets all fields of NON ArrayType in a dictionary. 
    nonArrayFieldsDictionary = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if not(isinstance(field.dataType, ArrayType))
    ])
    
    ######################################################################################    
    #Take all the column names from the dictionary nonArrayFieldsDictionary and store in a list
    #Use that list as the list of columns to select from the dataframe and write to the target
    #Note on the * operator here with *nonArrayColumnsList
        #The * operator unpacks an argument list. It allows you to call a function with the list items as individual arguments.
        #For instance, if sys.argv is ["./foo", "bar", "quux"], main(*sys.argv) is equivalent to main("./foo", "bar", "quux").
    ######################################################################################        
    nonArrayColumnsList = [col for col in nonArrayFieldsDictionary.keys()]    
    
    #Make a new dataframe that only contains the non array fields. This will include the json key field by default. 
    # This dataframe is then written to the target
    NonArrayFieldsDF = df.select(*nonArrayColumnsList)
    
    ######################################################################################
##TODO
    ## This is where the NonArrayFieldsDF is then written to the delta table. 
    ## The target table name will need to be dynamic based on the entity being processed & The array being processed. 
    ## Even if it is the top level array, use the column where the json came from as the array name. 
    
    #For now, just printing the schema and displaying the data to troubleshoot contents
    ######################################################################################
    
    #Troubleshooting to show the top level column was processed
    print("Processed: " + json_column_name)
    #only here for troubleshooting
    NonArrayFieldsDF.printSchema() 
    #only here for troubleshooting -see the dataframe that will be written to target in this iteration
    display(NonArrayFieldsDF)
    
    ######################################################################################
    #Prepare the dataframe for nested array processing
        #We will now drop the non array fields from the data frame to be left with the JSON Key field and any nested array fields
        #THen we will call this function recursively to process the array columns. 
    ######################################################################################
    
    #Before the nonArrayColumnsList list of columns is dropped from the dataframe, remove the JSON key field from the list. 
    #This ensures that the next nested array to be processed will have the KEY field of the parent JSON still included
    #This will be the relationship field from the seperate array entitty back to the parent JSON entity
    nonArrayColumnsList.remove(json_key_field_name)
    
    #Now we need to drop the non array fields from the main data frame since they have already been processed, leaving only the arrays still to process
    #Note, the KEY field of the parent json will not be dropped since it was removed from the nonArrayColumnsList list
    AdditionalArrayColumnsToProcessDF = df.drop(*nonArrayColumnsList)

    #Troubleshooting
    #display(AdditionalArrayColumnsToProcessDF)

    ######################################################################################        
    # Run a foreach loop to call this same function once per field in the df remaining
    ######################################################################################
    
    #First we need to get a list of the array fields left to process
    #Add all columns from the remaining dataframe to a list - this will include the key column
    AdditionalArrayColumnsToProcessDF_ArrayColumns = AdditionalArrayColumnsToProcessDF.columns
    
    #Remove the key column from the list so that when we recursively call this function, we loop once per array column
    #If we dont do this, then it tries to call the recursive function for the key field as well, then it gets stuck in an infinite loop
    AdditionalArrayColumnsToProcessDF_ArrayColumns.remove(json_key_field_name)
    
    for arrayColumn in AdditionalArrayColumnsToProcessDF_ArrayColumns:
        #Send this function a dataframe with the json key column and one of the array fields columns
        currentLoopJSONColumnToProcessDF = AdditionalArrayColumnsToProcessDF.select(json_key_field_name, arrayColumn)
        execute_autoflatten_tangent(currentLoopJSONColumnToProcessDF, arrayColumn, json_key_field_name)
    
    return #can possibly not include this return command

def execute_tangent(inputDF, primary_partition_column, json_column_name, json_key_field_name, spark_session):
    '''
    Description:
    This function executes the flattening of json records in the dataframe
        :param inputDF: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :primary_partition_column: [type: string] name of primary partition column
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return unstd_df: contains flattened dataframe with unstandardized column name format
    '''
    json_df = get_json_df_tangent(inputDF, primary_partition_column, json_column_name, spark_session)
    unstd_df = execute_autoflatten_tangent(json_df, json_column_name, json_key_field_name)
    return unstd_df

In [0]:
resultDF = execute_tangent(rawJsonDF, None, "JSONColumn", "JSONColumn>_id", spark)
display(resultDF)


json_column_name JSONColumn is array type
Processed: JSONColumn
root
-- JSONColumn>_id: string (nullable = true)
-- JSONColumn>about: string (nullable = true)
-- JSONColumn>address: string (nullable = true)
-- JSONColumn>age: long (nullable = true)
-- JSONColumn>balance: string (nullable = true)
-- JSONColumn>company: string (nullable = true)
-- JSONColumn>email: string (nullable = true)
-- JSONColumn>eyeColor: string (nullable = true)
-- JSONColumn>favoriteFruit: string (nullable = true)
-- JSONColumn>gender: string (nullable = true)
-- JSONColumn>greeting: string (nullable = true)
-- JSONColumn>guid: string (nullable = true)
-- JSONColumn>index: long (nullable = true)
-- JSONColumn>isActive: boolean (nullable = true)
-- JSONColumn>latitude: double (nullable = true)
-- JSONColumn>longitude: double (nullable = true)
-- JSONColumn>name: string (nullable = true)
-- JSONColumn>phone: string (nullable = true)
-- JSONColumn>picture: string (nullable = true)
-- JSONColumn>registered: string (nullable = true)
-- JSONColumn>NestedAttributesTest>NestedAttributeFive: string (nullable = true)
-- JSONColumn>NestedAttributesTest>NestedAttributeFour: string (nullable = true)
-- JSONColumn>NestedAttributesTest>NestedAttributeOne: string (nullable = true)
-- JSONColumn>NestedAttributesTest>NestedAttributeSix: string (nullable = true)
-- JSONColumn>NestedAttributesTest>NestedAttributeThree: string (nullable = true)
-- JSONColumn>NestedAttributesTest>NestedAttributeTwo: string (nullable = true)

JSONColumn>_id,JSONColumn>about,JSONColumn>address,JSONColumn>age,JSONColumn>balance,JSONColumn>company,JSONColumn>email,JSONColumn>eyeColor,JSONColumn>favoriteFruit,JSONColumn>gender,JSONColumn>greeting,JSONColumn>guid,JSONColumn>index,JSONColumn>isActive,JSONColumn>latitude,JSONColumn>longitude,JSONColumn>name,JSONColumn>phone,JSONColumn>picture,JSONColumn>registered,JSONColumn>NestedAttributesTest>NestedAttributeFive,JSONColumn>NestedAttributesTest>NestedAttributeFour,JSONColumn>NestedAttributesTest>NestedAttributeOne,JSONColumn>NestedAttributesTest>NestedAttributeSix,JSONColumn>NestedAttributesTest>NestedAttributeThree,JSONColumn>NestedAttributesTest>NestedAttributeTwo
622b43c868725fca32d05127,Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.,"116 Stewart Street, Longoria, Alaska, 3997",20,"$2,387.61",GENMY,hughesslater@genmy.com,green,banana,male,"Hello, Hughes Slater! You have 1 unread messages.",8a81ca3a-0813-4902-820d-d58e9703ec04,0,true,-47.586921,22.966975,Hughes Slater,+1 (883) 494-2174,http://placehold.it/32x32,2022-02-03T10:33:31 -02:00,5,4,1,6,3,2
622b43c8d2984c9956b839d9,Sunt culpa commodo ullamco aliqua fugiat sit fugiat eu fugiat eiusmod amet esse. Consequat nostrud exercitation deserunt magna labore elit ipsum Lorem laboris cupidatat reprehenderit mollit magna velit. Duis esse cillum proident ad id excepteur. Dolore excepteur cillum duis commodo. Sint dolore voluptate in amet nostrud culpa qui consequat tempor incididunt eiusmod irure sit officia. Consequat cillum occaecat Lorem nulla nisi qui excepteur.,"894 Cambridge Place, Dexter, Utah, 8622",25,"$1,910.43",GEEKNET,burchconrad@geeknet.com,green,apple,male,"Hello, Burch Conrad! You have 7 unread messages.",f4b512ca-e362-48a5-a5a5-1b21013bbd4c,1,false,33.526889,-69.444991,Burch Conrad,+1 (863) 523-2711,http://placehold.it/32x32,2018-10-11T07:17:58 -02:00,5,4,1,6,3,2
622b43c806bb45d49234240b,Lorem aute eiusmod et Lorem quis ea. Duis velit anim minim aute magna eiusmod excepteur nulla ad velit nostrud. Amet laboris voluptate nostrud nulla do minim non mollit duis sunt officia sunt nisi reprehenderit.,"111 Beverly Road, Canby, Oklahoma, 4837",24,"$1,711.22",SKINSERVE,ellapacheco@skinserve.com,brown,apple,female,"Hello, Ella Pacheco! You have 6 unread messages.",6f752038-214b-4cd7-b6d0-14250292a1e3,2,false,-18.438374,-137.932995,Ella Pacheco,+1 (915) 430-3434,http://placehold.it/32x32,2015-08-13T05:28:29 -02:00,5,4,1,6,3,2
622b43c8858495221ea6ecda,Sit non veniam amet minim. Sint cillum excepteur et incididunt. Do reprehenderit ex elit ut excepteur nostrud. Commodo elit minim velit sint esse duis eu sint minim laborum mollit. Fugiat nulla consequat veniam nisi culpa sit do veniam. Sunt anim est culpa nisi anim mollit consequat. Qui dolor sint id minim irure cillum.,"372 Ainslie Street, Cornfields, Alabama, 6363",23,"$1,889.73",EBIDCO,estellarich@ebidco.com,brown,strawberry,female,"Hello, Estella Rich! You have 7 unread messages.",121b43bf-abc7-46bf-8ac3-5355fe1d8290,3,true,38.887249,-41.450973,Estella Rich,+1 (957) 478-3558,http://placehold.it/32x32,2018-03-04T12:10:29 -02:00,5,4,1,6,3,2
622b43c876311a36c1c9da33,Laborum eu deserunt commodo deserunt pariatur sit quis fugiat consequat cillum ex mollit dolore consequat. Officia irure cillum et do incididunt eiusmod laborum consequat laboris ex fugiat ullamco. Anim nisi velit adipisicing consectetur Lorem ut cupidatat.,"521 Stratford Road, Stewartville, Vermont, 6312",33,"$1,498.87",LOTRON,carpentersullivan@lotron.com,brown,apple,male,"Hello, Carpenter Sullivan! Yo

json_column_name JSONColumn>friends is array type
Processed: JSONColumn>friends
root
-- JSONColumn>_id: string (nullable = true)
-- JSONColumn>friends>id: long (nullable = true)
-- JSONColumn>friends>name: string (nullable = true)

JSONColumn>_id,JSONColumn>friends>id,JSONColumn>friends>name
622b43c868725fca32d05127,0,Katie Carey
622b43c868725fca32d05127,1,Roth Gould
622b43c868725fca32d05127,2,Trisha Lindsay
622b43c8d2984c9956b839d9,0,Charles Newman
622b43c8d2984c9956b839d9,1,Maude Robinson
622b43c8d2984c9956b839d9,2,Taylor Goodwin
622b43c806bb45d49234240b,0,Montoya Becker
622b43c806bb45d49234240b,1,Reilly Phelps
622b43c806bb45d49234240b,2,Reeves Tran
622b43c8858495221ea6ecda,0,Walker William


json_column_name JSONColumn>tags is array type
Processed: JSONColumn>tags
root
-- JSONColumn>_id: string (nullable = true)
-- JSONColumn>tags: string (nullable = true)

JSONColumn>_id,JSONColumn>tags
622b43c868725fca32d05127,non
622b43c868725fca32d05127,ex
622b43c868725fca32d05127,sint
622b43c868725fca32d05127,culpa
622b43c868725fca32d05127,aliqua
622b43c868725fca32d05127,esse
622b43c868725fca32d05127,pariatur
622b43c8d2984c9956b839d9,in
622b43c8d2984c9956b839d9,non
622b43c8d2984c9956b839d9,adipisicing


# Tangent Version - Embedded write to Delta Tables
NOTE - this version can only cater for one level of nested arrays. Not multiple levels of nested arrays.

**PROBLEM: The nested arrays dont have unique keys. How do i merge join without causing duplication?**
**Even if they had unique keys, how do i determine what those unique keys are dynamicly in the code in order to tell the merge statement to use it for the join?**

This version of the JSON auto flatten functions has code embedded to write the one or more datasets out to delta tables inside the functions. 

**Considerations**
1. The column that has the JSON data must contain an array in each record.

What these functions can cater for
* Receive a dataframe with a column with JSON data
* it expects the top level to be an array of JSON records. It will explode this once. 
  * So each array in each row of the dataframe will be exploded as each row contains an array. 
  * At this point, each array record is a new record in the dataframe, but structs and nested arrays remain unchanged. No data duplication will occur at this level. 
  * If it is not an array, dont explode and move on to the next step
* Then expand all the struct fields to flatten it - this wont cause any data duplication. Struct fields are nested scalar attributes.
* Write the parent level attributes and flattened struct attributes to a target. 
* For each array column remaining, loop through them again and recursively call this function to write that array to a target as well

In [0]:

from pyspark.sql.functions import col, explode_outer, from_json, lit, concat, explode, array
from pyspark.sql.types import StructType, ArrayType

def write_dataFrame_to_delta_table_tangent_embeddedDeltaWrite(pInputDF, pTargetDataLakeContainer, pTargetDataLakeDirectory, pTargetEntityName, pPrimaryKeyColumnName, pIngestionDateTimeStampUTC ):
    '''
    Description: 
    This function will take in all the arguements necesary to take a dataframe and write the contexts to a Delta Table hosted in a dedicated storage account. 
        :pInputDF: The data frame to be written to the delta table. 
        :pTargetDataLakeContainer: The container name in the storage account where the delta table is to be hosted. E.g. "silver"
        :pTargetDataLakeDirectory: The directory in the storage account where the delta table is to be hosted. E.g. "WorldWideImporters/Batch/wwi-dimstockitem"
        :pTargetEntityName: The name of the delta table, also should link to the source entity name. E.g. "wwi-dimstockitem"
        :pPrimaryKeyColumnName: THe name of the column in the dataframe which can uniquely identify a record - the primary key. E.g. "StockItemID"
        :pIngestionDateTimeStampUTC: The date and time that is associated with this source file landing in the storage account e.g. "202203031147"
    Requirements
        1. The mount point of the target location must be defined beforehand, this function will not attempt to mount the location. 
        2. The unput data frame needs to have a single key column (passed to pPrimaryKeyColumnName) that uniquely identifies a row. This is used in the MERGE statement into the delta table. 
        3. The input data frame MUST have a column called IngestionDateTimeStampUTC
    Considerations
        1. The MERGE statement will be standardised to join on a single Key column & the standard IngestionDateTimeStampUTC field. 
        2. The target Delta Table will be partitioned as standard on the IngestionDateTimeStampUTC eifld. 
    '''
    ######################################################################################
    ## Create the vTargetDeltaTablePath - the location inside the pTargetDataLakeContainer of the storage account where the delta table will store its files
    ## Uses the Tangent Standard mount point structure of /mnt/datalake_<lakehouseLayerName>
    ######################################################################################
    #Example values expected
    # "pTargetDataLakeContainer": "silver",
    # "pTargetDataLakeDirectory": "WorldWideImporters/Batch/wwi-dimstockitem"
    vTargetDeltaTablePath = '/mnt/datalake_' + pTargetDataLakeContainer + '/' +pTargetDataLakeDirectory

    #Check values
    print('vTargetDeltaTablePath: ' + vTargetDeltaTablePath)
    
    ######################################################################################
    ## Create a temporary view from the inputDF which is references by the dynamic SQL code
    ######################################################################################
    pInputDF.createOrReplaceTempView("transformedView")
    
    ######################################################################################
    ## Create vDeltaTableCreateStatement - the SQL statement that will create the delta table if it doesn't exist. 
    ######################################################################################
    #E.g. TargetEntityName": "wwi-dimstockitem",
    vDeltaTableCreateStatement = 'CREATE TABLE IF NOT EXISTS ' + ' \n' \
    + 'datalakehouse_' + pTargetDataLakeContainer + '.' + pTargetEntityName + ' \n' \
    + 'USING DELTA ' + '\n' \
    + 'LOCATION \'' + vTargetDeltaTablePath + '\' ' + '\n' \
    + 'PARTITIONED BY ( IngestionDateTimeStampUTC )' + '\n' \
    + 'AS' + '\n' \
    + 'SELECT * FROM transformedView'

    #Check final output
    print('vDeltaTableCreateStatement: ' + vDeltaTableCreateStatement)

    #Exsecute the SQL
    spark.sql(vDeltaTableCreateStatement)
    
    ######################################################################################
    ## Write a dynamic MERGE to take the input dataframe and write it to the target
    ## This will not modify any data if the table was created in this run. It will only edit data on subsequent runs of this entity. 
    ######################################################################################
    vMergeDataIntoDeltaTableSQLStatement = 'MERGE INTO datalakehouse_' + pTargetDataLakeContainer + '.' + pTargetEntityName + ' AS target' + ' \n' \
    + 'USING transformedView as source' + ' \n' \
    + 'ON target.' + pPrimaryKeyColumnName + ' = source.' + pPrimaryKeyColumnName + ' \n' \
    + 'AND target.IngestionDateTimeStampUTC = source.IngestionDateTimeStampUTC' + ' \n' \
    + 'WHEN MATCHED THEN UPDATE SET *' + ' \n' \
    + 'WHEN NOT MATCHED THEN INSERT *'
    
    #Check Statement
    print('vMergeDataIntoDeltaTableSQLStatement: ' + vMergeDataIntoDeltaTableSQLStatement)

    #Exsecute the SQL
    spark.sql(vMergeDataIntoDeltaTableSQLStatement)
    
    ######################################################################################
    ## Perform optimisations on the delta table = OPTMISE ZORDER BY 
    ######################################################################################
    #Create the string that is the SQL query to execute using the latest ingestion time stamp and the right entity name
    vOptimiseTableWithZOrderClauseSQL = 'OPTIMIZE datalakehouse_' + pTargetDataLakeContainer + '.' + pTargetEntityName + '\n' \
    + 'WHERE `IngestionDateTimeStampUTC` = ' + pIngestionDateTimeStampUTC + '\n' \
    + 'ZORDER BY `' + pPrimaryKeyColumnName + '`'

    #Check the statement to be executed
    print(vOptimiseTableWithZOrderClauseSQL)

    #Exsecute the SQL
    vOptimiseTableWithZOrderClauseSQL_resultDF = spark.sql(vOptimiseTableWithZOrderClauseSQL)

    # Check the output of the dynamic SQL query
    display(vOptimiseTableWithZOrderClauseSQL_resultDF)
    
    ######################################################################################
    ## Perform optimisations on the delta table = VACUUM 
    ######################################################################################
    #Create the string that is the SQL query to execute
    vVacuumTableSQLStatement = 'VACUUM datalakehouse_' + pTargetDataLakeContainer + '.' + pTargetEntityName + ' RETAIN 168 HOURS'

    #Check the statement to be executed
    print(vVacuumTableSQLStatement)

    #Exsecute the SQL
    vVacuumTableSQLStatement_resultDF = spark.sql(vVacuumTableSQLStatement)

    # Check the output of the dynamic SQL query
    display(vVacuumTableSQLStatement_resultDF)
    
    ######################################################################################
    ## DROP the temporary view created on the input dataframe - cleanup
    ######################################################################################
    vDropTemporaryViewSQLStatement = 'DROP VIEW transformedView'
    
    #Check Statement
    print('vDropTemporaryViewSQLStatement: ' + vDropTemporaryViewSQLStatement)

    #Exsecute the SQL
    spark.sql(vDropTemporaryViewSQLStatement)


def get_json_df_tangent_embeddedDeltaWrite(inputDF, primary_partition_column, json_column_name, spark_session):
    '''
    Description:
    This function provides the schema of json records and the dataframe to be used for flattening. If this doesnt happen, the source JSON String remains a string and cant be queries like JSON
        :param inputDF: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :primary_partition_column: [type: string] name of primary partition column
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return df: dataframe to be used for flattening
    '''
    inputDF = inputDF if primary_partition_column is None else inputDF.drop(primary_partition_column)
    # creating a column transformedJSON to create an outer struct
    df1 = inputDF.withColumn('transformed_json', concat(lit("""{"transformed_json" :"""), inputDF[json_column_name], lit("""}""")))
    json_df = spark_session.read.json(df1.rdd.map(lambda row: row.transformed_json))
    # get schema
    json_schema = json_df.schema
    
    #Return a dataframe with the orignal column name but with proper JSON typed data
    df = df1.drop(json_column_name)\
        .withColumn(json_column_name, from_json(col('transformed_json'), json_schema))\
        .drop('transformed_json')\
        .select(f'{json_column_name}.*', '*')\
        .drop(json_column_name)\
        .withColumnRenamed("transformed_json", json_column_name)
    return df

def execute_autoflatten_tangent_embeddedDeltaWrite(df, json_column_name, json_key_field_name, pTargetDataLakeContainer, pTargetDataLakeDirectory, pTargetEntityName, pPrimaryKeyColumnName, pIngestionDateTimeStampUTC):
    '''
    Description:
    This function executes the core autoflattening operation
        :param df: [type: pyspark.sql.dataframe.DataFrame] dataframe to be used for flattening - contains at least one column with JSON data
        :param json_column_name: [type: string] name of the column with json string
        :json_key_field_name: [type: string] Name of the JSON Field that is the key of the document. Is used for nested arrays that are extracted to relate back to the primary dataset. 
            Unique per row of the df dataframe
            Should be in the format of "json_column_name>json_key_field_name" e.g. "JSONCOlumn>id"
        :return: The one or more dataframes extracted from the JSON is written to Delta Tables
        Fields needed by write_dataFrame_to_delta_table_tangent_embeddedDeltaWrite
            :pTargetDataLakeContainer: The container name in the storage account where the delta table is to be hosted. E.g. "silver"
            :pTargetDataLakeDirectory: The directory in the storage account where the delta table is to be hosted. E.g. "WorldWideImporters/Batch/wwi-dimstockitem"
            :pTargetEntityName: The name of the delta table, also should link to the source entity name. E.g. "wwi-dimstockitem"
            :pPrimaryKeyColumnName: THe name of the column in the dataframe which can uniquely identify a record - the primary key. E.g. "StockItemID"
            :pIngestionDateTimeStampUTC: The date and time that is associated with this source file landing in the storage account e.g. "202203031147"
    '''
    ######################################################################################
    # Check if the JSON data in the json_column_name column in each record of the df data frame is an array type. 
        # If so, explode the array once to get a flat structure with parent scalar attributes, and Structs (nested scalar attributes) and Arrays still intact. 
        # No data duplication will occur at this level. 
    ######################################################################################
    if isinstance(df.schema[json_column_name].dataType, ArrayType):
        #Keep this here permanently, for run time troubleshooting, even in prod
        print("json_column_name " + json_column_name + " is array type")
        
        #Since the entire json_column_name is an array, explode it once
        df = df.withColumn(json_column_name, explode_outer(json_column_name))
    else:
        print("json_column_name " + json_column_name + " is NOT array type: ", df.schema[json_column_name].dataType)
        
    ######################################################################################
    # Get all fields of StructType in a nested_fields dictionary. 
    # The next step will loop through these columns and expand them to get a flat structure. 
    # The parent attribute that contained the nested structure will be prefixed to the column name like Person>Name
    # This will not cause data duplication
    ######################################################################################    
    nested_fields = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if isinstance(field.dataType, StructType)
    ])
    
    ######################################################################################
    # For each struct type field in nested_fields, flatten them. 
    # Repeat until all nested_fields are covered
    ######################################################################################    
    while nested_fields:
        # check if there are any elements in the nested_fields dictionary
        if nested_fields:
            # get a column name to process
            column_name = list(nested_fields.keys())[0]
            # if field belongs to a StructType, all child fields inside it are accessed
            # and are aliased with complete path to every child field
            if isinstance(nested_fields[column_name], StructType):
                unnested = [col(column_name + '.' + child).alias(column_name + '>' + child) for child in [ n.name for n in  nested_fields[column_name]]]
                df = df.select("*", *unnested).drop(column_name)
            
        # Now that df is updated with the above column being flattened, get a new list of the fields of StructType in a fresh nested_fields dictionary
        # so that the next while loop can check again. 
        nested_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, StructType) # or isinstance(field.dataType, ArrayType)
        ])
    
    ######################################################################################
    # At this point, the top level structs have been flattened, but there are still nested arrays. 
    # Need to take non-array fields list, write that data to a target and only keep the remaining array fields with the key field used for related the data back to the primary data
    # For each remaining array field, do the above exactly the same again
    ######################################################################################
    # Gets all fields of NON ArrayType in a dictionary. 
    nonArrayFieldsDictionary = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if not(isinstance(field.dataType, ArrayType))
    ])
    
    ######################################################################################    
    #Take all the column names from the dictionary nonArrayFieldsDictionary and store in a list
    #Use that list as the list of columns to select from the dataframe and write to the target
    #Note on the * operator here with *nonArrayColumnsList
        #The * operator unpacks an argument list. It allows you to call a function with the list items as individual arguments.
        #For instance, if sys.argv is ["./foo", "bar", "quux"], main(*sys.argv) is equivalent to main("./foo", "bar", "quux").
    ######################################################################################        
    nonArrayColumnsList = [col for col in nonArrayFieldsDictionary.keys()]    
    
    #Make a new dataframe that only contains the non array fields. This will include the json key field by default. 
    # This dataframe is then written to the target
    NonArrayFieldsDF = df.select(*nonArrayColumnsList)
    
    ######################################################################################
    ## The NonArrayFieldsDF is now written to the designated delta table. 
    ##

    ######################################################################################
    #pTargetEntityName needs to be altered to cater for multiple entities being generated. 
    #This logic will take the json_column_name value and append it to the standard pTargetEntityName
    #Making a new variable so as to not overwrite the original paramter sent in. 
    vTargetEntityName = pTargetEntityName + json_column_name
    
    #Call the function to write to the applicable delta table
    write_dataFrame_to_delta_table_tangent_embeddedDeltaWrite(NonArrayFieldsDF, pTargetDataLakeContainer, pTargetDataLakeDirectory, vTargetEntityName, pPrimaryKeyColumnName, pIngestionDateTimeStampUTC )
    
    #Troubleshooting to show the top level column was processed
    print("Processed: " + json_column_name)
    #only here for troubleshooting
    NonArrayFieldsDF.printSchema() 
    #only here for troubleshooting -see the dataframe that will be written to target in this iteration
    display(NonArrayFieldsDF)
    
    ######################################################################################
    #Prepare the dataframe for nested array processing
        #We will now drop the non array fields from the data frame to be left with the JSON Key field and any nested array fields
        #THen we will call this function recursively to process the array columns. 
    ######################################################################################
    
    #Before the nonArrayColumnsList list of columns is dropped from the dataframe, remove the JSON key field from the list. 
    #This ensures that the next nested array to be processed will have the KEY field of the parent JSON still included
    #This will be the relationship field from the seperate array entitty back to the parent JSON entity
    nonArrayColumnsList.remove(json_key_field_name)
    
    #Now we need to drop the non array fields from the main data frame since they have already been processed, leaving only the arrays still to process
    #Note, the KEY field of the parent json will not be dropped since it was removed from the nonArrayColumnsList list
    AdditionalArrayColumnsToProcessDF = df.drop(*nonArrayColumnsList)

    #Troubleshooting
    #display(AdditionalArrayColumnsToProcessDF)

    ######################################################################################        
    # Run a foreach loop to call this same function once per field in the df remaining
    ######################################################################################
    
    #First we need to get a list of the array fields left to process
    #Add all columns from the remaining dataframe to a list - this will include the key column
    AdditionalArrayColumnsToProcessDF_ArrayColumns = AdditionalArrayColumnsToProcessDF.columns
    
    #Remove the key column from the list so that when we recursively call this function, we loop once per array column
    #If we dont do this, then it tries to call the recursive function for the key field as well, then it gets stuck in an infinite loop
    AdditionalArrayColumnsToProcessDF_ArrayColumns.remove(json_key_field_name)
    
    for arrayColumn in AdditionalArrayColumnsToProcessDF_ArrayColumns:
        #Send this function a dataframe with the json key column and one of the array fields columns
        currentLoopJSONColumnToProcessDF = AdditionalArrayColumnsToProcessDF.select(json_key_field_name, arrayColumn)
        execute_autoflatten_tangent(currentLoopJSONColumnToProcessDF, arrayColumn, json_key_field_name)
    
    return #can possibly not include this return command

def execute_tangent_embeddedDeltaWrite(inputDF, primary_partition_column, json_column_name, json_key_field_name, spark_session):
    '''
    Description:
    This function executes the flattening of json records in the dataframe
        :param inputDF: [type: pyspark.sql.dataframe.DataFrame] input dataframe
        :primary_partition_column: [type: string] name of primary partition column
        :param json_column_name: [type: string] name of the column with json string
        :param spark_session: SparkSession object
        :return unstd_df: contains flattened dataframe with unstandardized column name format
    '''
    json_df = get_json_df_tangent_embeddedDeltaWrite(inputDF, primary_partition_column, json_column_name, spark_session)
    unstd_df = execute_autoflatten_tangent_embeddedDeltaWrite(json_df, json_column_name, json_key_field_name)
    return unstd_df

In [0]:
#Get raw data into a dataframe
#Ensure you specify the quote and esacpe characters. 
#Multiple is false by default. 

#Ensure only the column with the JSON data is selected to have the downstream code work
rawJsonDF = spark.read\
.format("csv")\
.options(header="true", escape='"', quote='"', multiline=False, inferSchema=True)\
.load("/mnt/datalake_rawdata/json-generator.com/MockJsonInCSV.csv")\
#.select("JSONColumn")

display(rawJsonDF)


RowNumber JSONColumn 1 [ { "_id": "622b43c868725fca32d05127", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 0, "guid": "8a81ca3a-0813-4902-820d-d58e9703ec04", "isActive": true, "balance": "$2,387.61", "picture": "http://placehold.it/32x32", "age": 20, "eyeColor": "green", "name": "Hughes Slater", "gender": "male", "company": "GENMY", "email": "hughesslater@genmy.com", "phone": "+1 (883) 494-2174", "address": "116 Stewart Street, Longoria, Alaska, 3997", "about": "Sit non proident Lorem laboris non id excepteur voluptate nulla fugiat excepteur eiusmod est. Sunt deserunt officia cillum incididunt dolore cillum ipsum eu. In est laboris occaecat id sint laborum dolore mollit qui anim elit adipisicing. Eu labore esse qui do dolor Lorem elit fugiat eu duis. Eiusmod in incididunt qui do labore consectetur irure ipsum aliqua incididunt ex tempor nulla adipisicing. Sunt esse commodo proident est id Lorem ea. Commodo mollit est officia esse nulla nisi tempor ullamco aliqua.\r\n", "registered": "2022-02-03T10:33:31 -02:00", "latitude": -47.586921, "longitude": 22.966975, "tags": [ "non", "ex", "sint", "culpa", "aliqua", "esse", "pariatur" ], "friends": [ { "id": 0, "name": "Katie Carey" }, { "id": 1, "name": "Roth Gould" }, { "id": 2, "name": "Trisha Lindsay" } ], "greeting": "Hello, Hughes Slater! You have 1 unread messages.", "favoriteFruit": "banana" }, { "_id": "622b43c8d2984c9956b839d9", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 1, "guid": "f4b512ca-e362-48a5-a5a5-1b21013bbd4c", "isActive": false, "balance": "$1,910.43", "picture": "http://placehold.it/32x32", "age": 25, "eyeColor": "green", "name": "Burch Conrad", "gender": "male", "company": "GEEKNET", "email": "burchconrad@geeknet.com", "phone": "+1 (863) 523-2711", "address": "894 Cambridge Place, Dexter, Utah, 8622", "about": "Sunt culpa commodo ullamco aliqua fugiat sit fugiat eu fugiat eiusmod amet esse. Consequat nostrud exercitation deserunt magna labore elit ipsum Lorem laboris cupidatat reprehenderit mollit magna velit. Duis esse cillum proident ad id excepteur. Dolore excepteur cillum duis commodo. Sint dolore voluptate in amet nostrud culpa qui consequat tempor incididunt eiusmod irure sit officia. Consequat cillum occaecat Lorem nulla nisi qui excepteur.\r\n", "registered": "2018-10-11T07:17:58 -02:00", "latitude": 33.526889, "longitude": -69.444991, "tags": [ "in", "non", "adipisicing", "veniam", "irure", "excepteur", "laboris" ], "friends": [ { "id": 0, "name": "Charles Newman" }, { "id": 1, "name": "Maude Robinson" }, { "id": 2, "name": "Taylor Goodwin" } ], "greeting": "Hello, Burch Conrad! You have 7 unread messages.", "favoriteFruit": "apple" }, { "_id": "622b43c806bb45d49234240b", "NestedAttributesTest":{ "NestedAttributeOne": "1", "NestedAttributeTwo": "2", "NestedAttributeThree": "3", "NestedAttributeFour": "4", "NestedAttributeFive": "5", "NestedAttributeSix": "6" }, "index": 2, "guid": "6f752038-214b-4cd7-b6d0-14250292a1e3", "isActive": false, "balance": "$1,711.22", "picture": "http://placehold.it/32x32", "age": 24, "eyeColor": "brown", "name": "Ella Pacheco", "gender": "female", "company": "SKINSERVE", "email": "ellapacheco@skinserve.com", "phone": "+1 (915) 430-3434", "address": "111 Beverly Road, Canby, Oklahoma, 4837", "about": "Lorem aute eiusmod et Lorem quis ea. Duis velit anim minim aute magna eiusmod excepteur nulla ad velit nostrud. Amet laboris voluptate nostrud nulla do minim non mollit duis sunt officia sunt nisi reprehenderit.\r\n", "registered": "2015-08-13T05:28:29 -02:00", "latitude": -18.438374, "longitude": -137.932995, "tags": [ "duis", "occaecat", "Lorem", "aute", "ut", "nisi", "enim" ], "friends": [ { "id": 0, "name": "Montoy

# Test the wrtire_dataFrame_to_delta_table_tangent_embeddedDeltaWrite function

In [0]:
#temp vIngestionDateTimeStampUTC
vIngestionDateTimeStampUTC = '202203281141'

#temp add the vIngestionDateTimeStampUTC value to the dataframe
rawJsonDF = rawJsonDF.withColumn("IngestionDateTimeStampUTC", lit(vIngestionDateTimeStampUTC))

#temp create the parameters
pInputDF = rawJsonDF
pTargetDataLakeContainer = 'silver'
pTargetDataLakeDirectory = 'json-generator.com/MockJSONData'
pTargetEntityName = 'MockJSONData'
pPrimaryKeyColumnName = 'RowNumber'
pIngestionDateTimeStampUTC = vIngestionDateTimeStampUTC

#Call the new function to write a dataframe out to delta table dynamicallyu
write_dataFrame_to_delta_table_tangent_embeddedDeltaWrite(pInputDF, pTargetDataLakeContainer, pTargetDataLakeDirectory, pTargetEntityName, pPrimaryKeyColumnName, pIngestionDateTimeStampUTC )

vTargetDeltaTablePath: /mnt/datalake_silver/json-generator.com/MockJSONData
vDeltaTableCreateStatement: CREATE TABLE IF NOT EXISTS 
datalakehouse_silver.MockJSONData 
USING DELTA 
LOCATION '/mnt/datalake_silver/json-generator.com/MockJSONData' 
PARTITIONED BY ( IngestionDateTimeStampUTC )
AS
SELECT * FROM transformedView
vMergeDataIntoDeltaTableSQLStatement: MERGE INTO datalakehouse_silver.MockJSONData AS target 
USING transformedView as source 
ON target.RowNumber = source.RowNumber 
AND target.IngestionDateTimeStampUTC = source.IngestionDateTimeStampUTC 
WHEN MATCHED THEN UPDATE SET * 
WHEN NOT MATCHED THEN INSERT *
OPTIMIZE datalakehouse_silver.MockJSONData
WHERE `IngestionDateTimeStampUTC` = 202203281141
ZORDER BY `RowNumber`

path,metrics
dbfs:/mnt/datalake_silver/json-generator.com/MockJSONData,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 1, List(minCubeSize(107374182400), List(0, 0), List(1, 4550), 0, List(0, 0), 0, null), 0, 1, 1, false)"


VACUUM datalakehouse_silver.MockJSONData RETAIN 168 HOURS

path
dbfs:/mnt/datalake_silver/json-generator.com/MockJSONData


vDropTemporaryViewSQLStatement: DROP VIEW transformedView

# Test the new auto flatten function that also writes to output

In [0]:
from pyspark.sql.functions import lit,col,date_format,unix_timestamp

#temp vIngestionDateTimeStampUTC
vIngestionDateTimeStampUTC = '202203281141'

#temp add the vIngestionDateTimeStampUTC value to the dataframe
rawJsonDF = rawJsonDF.withColumn("IngestionDateTimeStampUTC", lit(vIngestionDateTimeStampUTC))

#temp create the parameters
pInputDF = rawJsonDF
pTargetDataLakeContainer = 'silver'
pTargetDataLakeDirectory = 'json-generator.com/MockJSONData_Flattened'
pTargetEntityName = 'MockJSONData_Flattened'
pPrimaryKeyColumnName = '_id'
pIngestionDateTimeStampUTC = vIngestionDateTimeStampUTC

json_column_name = 'JSONColumn'
json_key_field_name = '_id'

#Call the new function to write a dataframe out to delta table dynamically
execute_autoflatten_tangent_embeddedDeltaWrite(rawJsonDF, json_column_name, json_key_field_name, pTargetDataLakeContainer, pTargetDataLakeDirectory, pTargetEntityName, pPrimaryKeyColumnName, pIngestionDateTimeStampUTC)

json_column_name JSONColumn is NOT array type: StringType
vTargetDeltaTablePath: /mnt/datalake_silver/json-generator.com/MockJSONData_Flattened
vDeltaTableCreateStatement: CREATE TABLE IF NOT EXISTS 
datalakehouse_silver.MockJSONData_FlattenedJSONColumn 
USING DELTA 
LOCATION '/mnt/datalake_silver/json-generator.com/MockJSONData_Flattened' 
PARTITIONED BY ( IngestionDateTimeStampUTC )
AS
SELECT * FROM transformedView
vMergeDataIntoDeltaTableSQLStatement: MERGE INTO datalakehouse_silver.MockJSONData_FlattenedJSONColumn AS target 
USING transformedView as source 
ON target._id = source._id 
AND target.IngestionDateTimeStampUTC = source.IngestionDateTimeStampUTC 
WHEN MATCHED THEN UPDATE SET * 
WHEN NOT MATCHED THEN INSERT *

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2323595686520664> in <module> 
 19 
 20 #Call the new function to write a dataframe out to delta table dynamically 
 ---> 21 execute_autoflatten_tangent_embeddedDeltaWrite ( rawJsonDF , json_column_name , json_key_field_name , pTargetDataLakeContainer , pTargetDataLakeDirectory , pTargetEntityName , pPrimaryKeyColumnName , pIngestionDateTimeStampUTC ) 

 <command-2323595686520660> in execute_autoflatten_tangent_embeddedDeltaWrite (df, json_column_name, json_key_field_name, pTargetDataLakeContainer, pTargetDataLakeDirectory, pTargetEntityName, pPrimaryKeyColumnName, pIngestionDateTimeStampUTC) 
 244 
 245 #Call the function to write to the applicable delta table 
 --> 246 write_dataFrame_to_delta_table_tangent_embeddedDeltaWrite ( NonArrayFieldsDF , pTargetDataLakeContainer , pTargetDataLakeDirectory , vTargetEntityName , pPrimaryKeyColumnName , pIngestionDateTimeStampUTC ) 
 247 
 248 #Troubleshooting to show the top level column was processed 

 <command-2323595686520660> in write_dataFrame_to_delta_table_tangent_embeddedDeltaWrite (pInputDF, pTargetDataLakeContainer, pTargetDataLakeDirectory, pTargetEntityName, pPrimaryKeyColumnName, pIngestionDateTimeStampUTC) 
 70 
 71 #Exsecute the SQL 
 ---> 72 spark . sql ( vMergeDataIntoDeltaTableSQLStatement ) 
 73 
 74 ###################################################################################### 

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 775 [ Row ( f1 = 1 , f2 = 'row1' ) , Row ( f1 = 2 , f2 = 'row2' ) , Row ( f1 = 3 , f2 = 'row3' ) ] 
 776 """
 --> 777 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 778 
 779 def table ( self , tableName ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : cannot resolve `target._id` in search condition given columns target.`RowNumber`, target.`JSONColumn`, target.`IngestionDateTimeStampUTC`, source.`RowNumber`, source.`JSONColumn`, source.`IngestionDateTimeStampUTC`; line 3 pos 3